In [ ]:
X = last_train_data.drop(["id", "country_destination"], axis=1)
y = last_train_data["country_destination"]
x_new = last_test_data.drop(["id", "country_destination"], axis=1)

model1 = XGBClassifier(nthread=3)
model2 = ExtraTreesClassifier(n_estimators=250, random_state=0)
model3 = LinearDiscriminantAnalysis(n_components=3, solver="svd", store_covariance=True)
ensemble = VotingClassifier(estimators=[('xg', model1), ('tree', model2), ('lda', model3)], 
                            voting='soft', weights=[1, 1, 2])

probas = [c.fit(X, y).predict_proba(x_new) for c in (model1, model2, model3, ensemble)]
class1_1 = [pr[0, 0] for pr in probas]
class2_1 = [pr[0, 1] for pr in probas]


ind = np.arange(4)
width = 0.35  # bar width
p1 = plt.bar(ind, np.hstack(([class1_1[:-1], [0]])), width, color='green')
p2 = plt.bar(ind + width, np.hstack(([class2_1[:-1], [0]])), width, color='lightgreen')
p3 = plt.bar(ind, [0, 0, 0, class1_1[-1]], width, color='blue')
p4 = plt.bar(ind + width, [0, 0, 0, class2_1[-1]], width, color='steelblue')

plt.xticks(ind + 0.5 * width, ['LogisticRegression\nweight 1',
                               'QDA\nweight 1',
                               'GaussianNB\nweight 2',
                               'VotingClassifier'])
plt.ylim([0, 1.1])
plt.title('Class probabilities for sample 1 by different classifiers')
plt.legend([p1[0], p2[0]], ['class 1', 'class 2'], loc='upper left')
plt.show()

In [ ]:
pro_t = ensemble.predict_proba(last_test_data.drop(["id", "country_destination"], axis=1))

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(sample_submission_NDF)):
    idx = sample_submission_NDF.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(pro_t[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('lilil_t.csv', sep=',', na_rep='NaN', index = False)